In [11]:
import json

DB = []
INDEXES = {'given_name': {}}

def add_row(row):
  row_id = len(DB)
  for index in INDEXES:
    if index in row:
      value = row[index]
      if value in INDEXES[index]:
        INDEXES[index][value].add(row_id)
      else:
        INDEXES[index][value] = {row_id}
  DB.append(row)
  
def query(field, value):
  if field in INDEXES:
    result = []
    for row_id in INDEXES[field][value]:
      result.append(DB[row_id])
  else:
    result = [row for row in DB if row[field] == value]
  return result

def save(filename='data.json'):
  open(filename, 'w').write(json.dumps(DB))
  
def load(filename='data.json'):
  for row in json.loads(open(filename, 'r').read()):
    add_row(row)
  
add_row({'family_name': 'Einstein', 'given_name': 'Albert'})
add_row({'family_name': 'Franklin', 'given_name': 'Benjamin'})
add_row({'family_name': 'Lovelace', 'given_name': 'Ada'})
add_row({'family_name': 'Atkinson', 'given_name': 'Rowan'})

add_row({'family_name': 'de Jong', 'given_name': 'Albert'})
query('given_name', 'Albert')

open = 5


print(open)


5


In [0]:
#@title Basics  {display-mode: "form"}


DB = []
INDEXES = {'given_name': {}}

def add_row(row):
  row_id = len(DB)
  DB.append(row)
  for index in INDEXES:
    if index in row:
      INDEXES[index].setdefault(row[index], set()).add(row_id)
  
def get_row(index):
  return DB[index]

def query(field, value):
  if field in INDEXES:
    result = []
    for row_id in INDEXES[field].get(value, []):
      result.append(DB[row_id])
  else:
    result = [row for row in DB if row[field] == value]
  return result

add_row({'family_name': 'Einstein', 'given_name': 'Albert'})
add_row({'family_name': 'Franklin', 'given_name': 'Benjamin'})
add_row({'family_name': 'Lovelace', 'given_name': 'Ada'})
add_row({'family_name': 'Atkinson', 'given_name': 'Rowan'})

query('given_name', 'Albert')

In [14]:

import sqlite3

conn = sqlite3.connect(":memory:")
c = conn.cursor()

c.execute('CREATE TABLE names (id integer, family_name text, given_name text)')
c.execute('INSERT INTO names VALUES (?, ?, ?)', [1, 'Einstein', 'Albert'])
c.execute('INSERT INTO names VALUES (?, ?, ?)', [2, 'Franklin', 'Benjamin'])
c.execute('INSERT INTO names VALUES (?, ?, ?)', [3, 'Lovelace', 'Ada'])
c.execute('INSERT INTO names VALUES (?, ?, ?)', [4, 'Atkinson', 'Rowan'])
c.execute('INSERT INTO names VALUES (?, ?, ?)', [5, 'de Jong', 'Albert']) 

c.execute('CREATE INDEX given_name_index ON names (given_name)')

for row in c.execute('SELECT * FROM names WHERE given_name = ?', ['Albert']):
  print(row)
  

(1, 'Einstein', 'Albert')
(5, 'de Jong', 'Albert')


In [0]:
#@title SQLite index {display-mode: "form"}

c.execute('CREATE INDEX given_name_index ON names (given_name)')

In [16]:


c.execute('DROP TABLE IF EXISTS birth_dates')
c.execute('CREATE TABLE birth_dates (name_id integer, birthdate text)')

for name_id, birthdate in enumerate(['03/14/1879', '01/17/1706', '12/10/1815', '01/6/1955']):
  c.execute('INSERT INTO birth_dates VALUES (?, ?)', [name_id, birthdate])
  
for row in c.execute('''
  SELECT id, family_name, given_name, birthdate 
  FROM names 
  LEFT JOIN birth_dates 
  ON names.id = birth_dates.name_id 
  WHERE given_name = ?''',  ['Albert']):
  print(row)

(1, 'Einstein', 'Albert', '01/17/1706')
(5, 'de Jong', 'Albert', None)


https://blog.jooq.org/2016/07/05/say-no-to-venn-diagrams-when-explaining-joins/